In [ ]:
# Author: Karl Byland

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
#Used to save the model after training
import sklearn

import joblib
print(sklearn.__version__)


1.7.1


In [ ]:
import sys
sys.path.append(r"C:\DIT826\team1")
print(sys.path)

import os
os.getcwd()
import sentio
import sentio.settings.development
os.environ.setdefault(
    "DJANGO_SETTINGS_MODULE",
    "sentio.settings.development"
)


['C:\\Users\\Waggi\\.conda\\envs\\SEAI\\python313.zip', 'C:\\Users\\Waggi\\.conda\\envs\\SEAI\\DLLs', 'C:\\Users\\Waggi\\.conda\\envs\\SEAI\\Lib', 'C:\\Users\\Waggi\\.conda\\envs\\SEAI', '', 'C:\\Users\\Waggi\\AppData\\Roaming\\Python\\Python313\\site-packages', 'C:\\Users\\Waggi\\.conda\\envs\\SEAI\\Lib\\site-packages', 'C:\\Users\\Waggi\\.conda\\envs\\SEAI\\Lib\\site-packages\\win32', 'C:\\Users\\Waggi\\.conda\\envs\\SEAI\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\Waggi\\.conda\\envs\\SEAI\\Lib\\site-packages\\Pythonwin', 'C:\\DIT826\\team1']


'sentio.settings.development'

In [4]:
import django

try:
    django.setup()
except Exception as e:
    import traceback
    traceback.print_exc()

In [5]:
print(sys.path)
import os
import django


os.environ.setdefault("DJANGO_SETTINGS_MODULE", "team1.sentio.settings")


['C:\\Users\\Waggi\\.conda\\envs\\SEAI\\python313.zip', 'C:\\Users\\Waggi\\.conda\\envs\\SEAI\\DLLs', 'C:\\Users\\Waggi\\.conda\\envs\\SEAI\\Lib', 'C:\\Users\\Waggi\\.conda\\envs\\SEAI', '', 'C:\\Users\\Waggi\\AppData\\Roaming\\Python\\Python313\\site-packages', 'C:\\Users\\Waggi\\.conda\\envs\\SEAI\\Lib\\site-packages', 'C:\\Users\\Waggi\\.conda\\envs\\SEAI\\Lib\\site-packages\\win32', 'C:\\Users\\Waggi\\.conda\\envs\\SEAI\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\Waggi\\.conda\\envs\\SEAI\\Lib\\site-packages\\Pythonwin', 'C:\\DIT826\\team1']


'sentio.settings.development'

In [6]:
import os
print(os.listdir(r"C:\DIT826"))


['team1']


In [7]:
from apps.ml_admin.models import DatasetRecord
from asgiref.sync import sync_to_async
# Load data training data into a pandas dataframe


@sync_to_async
def extract_data_async():

    qs = DatasetRecord.objects.all()
    print(qs.exists())  # True if there are any records
    print(qs.count())   # Number of records

    qs = DatasetRecord.objects.all().values(
        "text",
        "label",
        "category_id"
    )

    #df = pd.DataFrame(list(qs))
    df = pd.DataFrame(list(DatasetRecord.objects.all().values()))
    df.columns

    # Ensure numeric category aligns with the text label
    label_map = {
        "normal": 0,
        "depression": 1,
        "suicidal": 2,
        "stress": 3,
    }
    #df["status"] = df["label"].map(label_map)

    return df




df = await extract_data_async()

df.head()



True
50590


,id,text,label,category_id,normal,depression,suicidal,stress,dataset_type,data_upload_id,imported_at
0,1,oh my gosh,Stress,2,0,0,0,1,train,7,2025-12-01 15:44:25.944463+00:00
1,2,"trouble sleeping, confused mind, restless hear...",Stress,2,0,0,0,1,train,7,2025-12-01 15:44:25.944463+00:00
2,3,"All wrong, back off dear, forward doubt. Stay ...",Stress,2,0,0,0,1,train,7,2025-12-01 15:44:25.944463+00:00
3,4,I've shifted my focus to something else but I'...,Stress,2,0,0,0,1,train,7,2025-12-01 15:44:25.945475+00:00
4,5,"I'm restless and restless, it's been a month n...",Stress,2,0,0,0,1,train,7,2025-12-01 15:44:25.945475+00:00


In [ ]:
label_map = {
    "normal": 0,
    "depression": 1,
    "suicidal": 2,
    "stress": 3,
}

df['label'] = df['label'].str.lower()  # convert all labels to lowercase
df['status'] = df['label'].map(label_map)
df['status'] = df['status'].astype(int)
df['text'] = df['text'].astype(str)
#Makes the text all lowercase
df['text'] = df['text'].astype(str).str.lower()
#Drops the rows where the text is missing
df = df.dropna(subset=['label'])

train_df = df.sample(frac=0.75, random_state=42)
test_df  = df.drop(train_df.index)

X_train = train_df['text']
y_train = train_df['label']

In [9]:
model = Pipeline([
    ("tfidf", TfidfVectorizer(
        stop_words="english",
        ngram_range=(1,2),
        min_df=2,
        max_df=0.95
    )),
    ("clf", LogisticRegression(class_weight="balanced", max_iter=1000))
])


In [10]:
model.fit(X_train, y_train)


,steps,"[('tfidf', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [11]:
save_path = "ml_pipeline/toy_models/LRmodel.pkl"
#Saves the trained model to mode.pk1
joblib.dump(model, "LRmodel.pkl")



['LRmodel.pkl']

In [12]:
#Extracts the text and label for the test data frame
X_test = test_df['text']
y_test = test_df['label']
y_pred = model.predict(X_test)


In [13]:
metrics = classification_report(y_test, y_pred, output_dict=True)
accuracy = accuracy_score(y_test, y_pred)


print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))


Accuracy: 0.7833649588867805
              precision    recall  f1-score   support

  depression       0.77      0.65      0.70      3794
      normal       0.87      0.91      0.89      3722
      stress       0.81      0.84      0.82      2423
    suicidal       0.67      0.75      0.70      2709

    accuracy                           0.78     12648
   macro avg       0.78      0.79      0.78     12648
weighted avg       0.78      0.78      0.78     12648



In [15]:
from apps.ml_admin.models import ModelVersion
from django.contrib.auth.models import User

@sync_to_async
def get_user():
    user = User.objects.first()

@sync_to_async
def create_model_version():
    ModelVersion.objects.create(
    version_name="v1.2",
    model_file_path=save_path,
    accuracy=accuracy,
    precision=float(metrics["depression"]["precision"]),
    recall=float(metrics["depression"]["recall"]),
    f1_score=float(metrics["depression"]["f1-score"]),
    is_active=True,
    created_by=user
    )

user = await get_user()
print(user)
await create_model_version()


None


In [16]:
@sync_to_async
def test():
    ModelVersion.objects.filter(version_name="v1.0").exists()  # True if created
trying = await test()
print(trying)



None


In [18]:
sample = ["I am happy"]
print(model.predict(sample))


['normal']


In [ ]:
#    "Normal": 0,
#   "Depression": 1,
#   "Suicidal": 2,
#   "Anxiety": 3,
#   "Stress": 4,
#   "Bipolar": 5,
#   "Personality disorder": 6